# Comparaison de la taille du modèle Mistral-7B-v0.1 sauvegardé sur le disque

Dans ce notebook, nous allons comparer la taille du modèle **Mistral-7B-v0.1** sauvegardé sur le disque, modèle pré-entraîné et modèle fine-tuné. On utiisera la **quantization** pour l'utilisation du modèle et la méthode **LoRA** pour le fine-tuning. Les étapes suivantes seront suivies :

1. Récupération du modèle Mistral-7B-v0.1.
2. Sauvegarde du modèle Mistral-7B-v0.1 sur le disque.
3. Fine-tuning du modèle Mistral-7B-v0.1.
4. Sauvegarde du modèle Mistral-7B-v0.1 fine-tuné sur le disque.
5. Comparaison des tailles.

## Récupération du modèle Mistral-7B-v0.1

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
# - Le GPU est utilisé ici par défaut, pas besoin de charger le module CUDA
# - Ne pas lancer ce bout de codes plusieurs fois, sinon : ValueError: Some modules are dispatched on the CPU or the disk...

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="auto")

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

## Sauvegarde du modèle Mistral-7B-v0.1 sur le disque

In [3]:
# Spécification du répertoire de sauvegarde
save_directory = "../models/mistral-7b-v0.1"

# Sauvegarde du modèle
model.save_pretrained(save_directory)

## Fine-tuning du modèle Mistral-7B-v0.1

Pour le fine-tuning du modèle Mistral-7B-v0.1, consulter le notebook `mistral_finetune_own_data.ipynb`.

## Sauvegarde du modèle Mistral-7B-v0.1 fine-tuné sur le disque

Pour la sauvegarde du modèle Mistral-7B-v0.1 fine-tuné sur le disque, consulter le notebook `mistral_finetune_own_data.ipynb`.

## Comparaison des tailles

La sauvegarde d'un modèle est réalisée avec deux types de fichiers :

- un (ou des) fichier(s) de configuration `.json`.
- un fichier du modèle `.safetensors`.

La majorité de l'information du modèle est contenue dans le fichier `.safetensors`, étant donné que le(s) fichier(s) `.json` sont simplement là pour la configuration.

On observe alors la taille des fichiers `.safetensors` :

- modèle pré-entraîné : 4 125 687 623 octets
- modèle fine-tuné : 864 513 616 octets

Le rapport de taille vaut :

$$
\frac{864 513 616}{4 125 687 623} = 0.2095
$$

Le modèle fine-tuné occupe donc 80 % d'emplacement mémoire en moins que le modèle pré-entaîné !